# Data pool の操作 

## 1. スケールアウトテーブルの作成

In [10]:
USE [DataPool];

-- オブジェクトの初期化
IF EXISTS (SELECT * FROM sys.external_tables WHERE name = 'clickstream_datamart')
BEGIN
	DROP EXTERNAL TABLE clickstream_datamart
END;
GO

Commands completed successfully.

Total execution time: 00:00:00.6171997

**1. Data Pool に、スケールアウトデータマート用のテーブルを作成**  
本クエリにより、Data Pool 内の各 SQL Server にテーブルが作成される  
(テーブル作成が Data Pool の各 SQL Server に伝搬されるまで多少のタイムラグがある)

作成されたテーブルは列ストアインデックスが自動的に作成された状態となっている

In [12]:
USE [DataPool];

-- テーブルの作成
CREATE EXTERNAL TABLE [clickstream_datamart]
(
	wcs_click_date_sk BIGINT , 
	wcs_click_time_sk BIGINT , 
	wcs_sales_sk BIGINT , 
	wcs_item_sk BIGINT , 
	wcs_web_page_sk BIGINT , 
	wcs_user_sk BIGINT
) 
WITH
( 
	DATA_SOURCE = SqlDataPool,	
	DISTRIBUTION = ROUND_ROBIN
);
GO
WAITFOR DELAY '00:00:10'

Commands completed successfully.

Total execution time: 00:00:00.6103302

Commands completed successfully.

Total execution time: 00:00:10.0023408

**2. リンクサーバー経由で確認**  
Master Instance に設定したリンクサーバー経由で、Data Pool 内の SQL Server の情報を確認

In [11]:
USE [DataPool];

SELECT
	(SELECT name from [DATA-0-0.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.sys.servers WHERE server_id = 0) AS server_name
	, o.name
	, i.name
	, i.type_desc
	, (SELECT COUNT(*) FROM [DATA-0-0.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.dbo.clickstream_datamart) AS count
FROM
	[DATA-0-0.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.sys.indexes AS i
	LEFT JOIN 
	[DATA-0-0.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.sys.objects AS o
	ON i.object_id = o.object_id
WHERE
	o.name = 'clickstream_datamart'
UNION
SELECT
	(SELECT name from [DATA-0-1.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.sys.servers WHERE server_id = 0) AS server_name
	, o.name
	, i.name
	, i.type_desc
	, (SELECT COUNT(*) FROM [DATA-0-1.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.dbo.clickstream_datamart) AS count
FROM
	[DATA-0-1.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.sys.indexes AS i
	LEFT JOIN 
	[DATA-0-1.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.sys.objects AS o
	ON i.object_id = o.object_id
WHERE
	o.name = 'clickstream_datamart'
GO

: Msg 7314, Level 16, State 1, Line 3
The OLE DB provider "MSOLEDBSQL" for linked server "DATA-0-0.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL" does not contain the table ""DataPool"."dbo"."clickstream_datamart"". The table either does not exist or the current user does not have permissions on that table.

Total execution time: 00:00:00.0231298

![Linked Server](https://github.com/MasayukiOzawa/decode-2019-demo/raw/master/Images/03.Data%20Store/02.Data%20Pool/Data%20Pool%20Database%20object.png)

## 2. スケールアウトデータマートにデータの投入

**1. データ投入**  
INSERTO SELECT でスケールアウトデータマートのテーブルを指定することで、データマートにデータを投入することができる。  

In [14]:
USE [DataPool];
INSERT INTO clickstream_datamart
SELECT * FROM StoragePool..StoragePoolTBL
GO

(1497 rows affected)

Total execution time: 00:00:03.5751605

**2. 投入されたデータの確認**  
リンクサーバー経由で投入されたデータの状況を確認

In [15]:
USE [DataPool];

SELECT
	(SELECT name from [DATA-0-0.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.sys.servers WHERE server_id = 0) AS server_name
	, o.name
	, i.name
	, i.type_desc
	, (SELECT COUNT(*) FROM [DATA-0-0.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.dbo.clickstream_datamart) AS count
FROM
	[DATA-0-0.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.sys.indexes AS i
	LEFT JOIN 
	[DATA-0-0.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.sys.objects AS o
	ON i.object_id = o.object_id
WHERE
	o.name = 'clickstream_datamart'
UNION
SELECT
	(SELECT name from [DATA-0-1.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.sys.servers WHERE server_id = 0) AS server_name
	, o.name
	, i.name
	, i.type_desc
	, (SELECT COUNT(*) FROM [DATA-0-1.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.dbo.clickstream_datamart) AS count
FROM
	[DATA-0-1.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.sys.indexes AS i
	LEFT JOIN 
	[DATA-0-1.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.sys.objects AS o
	ON i.object_id = o.object_id
WHERE
	o.name = 'clickstream_datamart'
GO

(2 rows affected)

Total execution time: 00:00:00.1560458

server_name,name,name,type_desc,count
data-0-0,clickstream_datamart,cci,CLUSTERED COLUMNSTORE,1181
data-0-1,clickstream_datamart,cci,CLUSTERED COLUMNSTORE,316


**3. スケールアウトデータマーのデータ検索**  
スケールアウトデータマートに投入されているデータは、Master Instance からデータを取得することができる。

In [16]:
USE [DataPool];

SELECT TOP 25 * FROM clickstream_datamart

(25 rows affected)

Total execution time: 00:00:02.5348125

wcs_click_date_sk,wcs_click_time_sk,wcs_sales_sk,wcs_item_sk,wcs_web_page_sk,wcs_user_sk
38584,53740,NULL,17798,18,NULL
38584,42373,NULL,11965,18,NULL
38584,60657,NULL,10185,18,NULL
38584,13440,NULL,15478,18,NULL
38584,78804,NULL,10026,18,NULL
38584,86159,NULL,245,18,NULL
38584,79364,NULL,5722,18,NULL
38584,86173,NULL,4495,18,NULL
38584,82380,NULL,17036,18,NULL
38584,37842,NULL,10401,18,NULL


![Scale Out Data mart](https://raw.githubusercontent.com/MasayukiOzawa/decode-2019-demo/master/Images/03.Data%20Store/02.Data%20Pool/Scale-out%20data%20mart%20select.png)